In [5]:
from bs4 import BeautifulSoup as bsoup
from urllib.request import urlopen 
import pandas as pd 
import re
import requests

# setting lists for variables of interest
version = []
make = []
model =[]
Year = []
Mile = []
Price = []
Seller_type = []

mydf = pd.DataFrame()

for page in range(1,20):
    url = 'https://www.lacentrale.fr/listing?makesModelsCommercialNames=RENAULT%3AZOE&options=&page='+str(page)+'&regions=FR-IDF'
    client = urlopen(url)
    page_client = client.read()
    

    page_soup = bsoup(page_client,'html.parser')
    containers = page_soup.findAll('div',attrs={'class','contour'})

    for container in containers:
    
        cont_version = container.div.img['alt']
        version.append(cont_version[12:]) # intense
    
    
        cont_brand =container.find('span',attrs={'class','brandModel txtGrey3'})
        make.append(cont_brand.text[:7]) 
        model.append(cont_brand.text[8:]) 
    
        cont_year = container.findAll('div',attrs={'class','fieldYear'})
        year = cont_year[0].text
        Year.append(year)
    
    
        cont_mileage = container.findAll('div',attrs={'class','fieldMileage'})
        mile = cont_mileage[0].text
        mileage_clean = re.findall(r'\b\d+', mile)
        mileage_final = tuple(mileage_clean)[0] + tuple(mileage_clean)[-1]
        Mile.append(mileage_final)

    
        cont_price= container.findAll('div',attrs={'class','fieldPrice sizeC'})
        price = cont_price[0].text
        price_clean = re.findall(r'\b\d+', price)
        price_merged = tuple(price_clean)[0]+tuple(price_clean)[1]
        Price.append(price_merged)
    
        cont_seller= container.find('p',attrs={'class',"txtBlack typeSeller hiddenPhone"})
        Seller_type.append(cont_seller.text)
    

mydf['make'] = make
mydf['version'] = version
mydf['model'] = model
mydf['Year'] = Year
mydf['mileage'] = Mile
mydf['Price'] = Price
mydf['Seller_type'] = Seller_type

In [6]:
url_argus = "https://www.lacentrale.fr/cote-auto-{}-{}-{}-{}.html"

def build_soup_from_url(url):
    page = requests.get(url)
    html = page.text
    soup = bsoup(html,"html.parser")
    return soup

def get_argus(brand, model, version, Year):
  
    url = url_argus.format(brand.lower(), model.lower(), version.lower().replace(" ", "+"), Year)
    soup = build_soup_from_url(url)
    argus = soup.find("span", class_ = "jsRefinedQuot")
    if argus is None:
        argus = 0
    else:
        argus = argus.getText().replace(" ", "").replace("€","")
    return argus

In [7]:
argus_list = []
for i in range(len(mydf)):
    brand = (mydf['make'][i])
    model = (mydf['model'][i])
    version = (mydf['version'][i])
    Year = (mydf['Year'][i])
    argus_list.append(get_argus(brand = brand,model = model,version= version,Year = Year))

mydf['argus'] = argus_list
 

In [9]:
import numpy as np
mydf = mydf[(mydf[['argus']] != 0).all(axis=1)]

# Calculate % difference between indicated price and argus price.
mydf['%_difference'] = ((mydf['Price']).astype(float)/(mydf['argus']).astype(float))*100


# Global overview of buying and selling opportunities based on argus prices.
mydf['reco'] = np.where(mydf['%_difference']<100, 'buy', 'sell')
print(mydf.reco.value_counts())

buy     76
sell    67
Name: reco, dtype: int64
